# Pensar Contábil

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    tbl_results = soup.find('table', {'class': 'listing'})
    a_links = []
    for a in tbl_results.find_all('a'):
           if 'Resumo' in a.text:
                a_links.append(a['href'])
    return a_links



In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links



In [5]:
url_search_text = 'http://www.atena.org.br/revista/ojs-2.2.3-06/index.php/pensarcontabil/search/advancedResults?query=__query__&searchJournal=&author=&title=&fullText=&supplementaryFiles=&discipline=&subject=&type=&coverage=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'

links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[2]

link

'http://www.atena.org.br/revista/ojs-2.2.3-06/index.php/pensarcontabil/article/view/3655'

In [6]:
links

['http://www.atena.org.br/revista/ojs-2.2.3-06/index.php/pensarcontabil/article/view/2328',
 'http://www.atena.org.br/revista/ojs-2.2.3-06/index.php/pensarcontabil/article/view/2380',
 'http://www.atena.org.br/revista/ojs-2.2.3-06/index.php/pensarcontabil/article/view/3655',
 'http://www.atena.org.br/revista/ojs-2.2.3-06/index.php/pensarcontabil/article/view/3630',
 'http://www.atena.org.br/revista/ojs-2.2.3-06/index.php/pensarcontabil/article/view/2620']

In [7]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [9]:
def captura_titulo(soup):
    try:
        title_tag = soup.find('div', {'id': 'content'})
        title = title_tag.find('h3').text
    except:
        title = ''
    return {'Título': title}

In [10]:
captura_titulo(soup)['Título']

'Análise de Indicadores de Desempenho da Gestão Ambiental em Mineradoras no Brasil'

In [11]:
def captura_data(soup):
    try:
        year_tags = soup.find('div', {'id': 'breadcrumb'})
        year_tag = year_tags.find_all('a')
        year_text = year_tag[1].text
        year = year_text[-5:-1]
    except:
        year = ''
    return {'Data de Publicação': year}

In [12]:
captura_data(soup)

{'Data de Publicação': '2021'}

In [13]:
def captura_resumo(soup):
    try:
        content_tag = soup.find('div', {'id': 'content'})
        divs = content_tag.find_all('div')
        try:
            ps = divs[3].find_all('p')
            abstract_full = ps[2].text
            abstract = abstract_full.replace('RESUMO', '')
        except:
            abstract = divs[3].text
    except:
        abstract = ''
    return {'Resumo': abstract}

In [14]:
captura_resumo(soup)

{'Resumo': 'O desenvolvimento sustentável é requisito fundamentalpara a sobrevivência humana e as empresas, como causadorasde impactos, necessitam se enquadrar em diversosaspectos a fim de evitar ou amenizar problemas ambientaise sociais. O setor de mineração é relevante para a economiabrasileira, mas acarreta sérios prejuízos ao meio ambiente,principalmente por sua rigidez locacional, e os impactos ealterações causados devem ser alvo de controle. Dessa forma,a presente pesquisa objetivou analisar os relatórios desustentabilidade das mineradoras do Brasil, por meio de indicadoresde desempenho da gestão ambiental, identificandoinformações contábeis voluntárias relevantes, utilizando indicadorespara mensurar o nível de evidenciação dessas informaçõese comparando os resultados dessas empresas nodecorrer dos anos. Concluiu-se que, por não ser obrigatório,a maioria das empresas não divulgam relatórios de sustentabilidadee aquelas que divulgam precisam melhorar bastanteseus níveis de evidenc

In [15]:
def captura_palavras_chave(soup):
    content_tag = soup.find('div', {'id': 'content'})
    divs = content_tag.find_all('div')
    try:
        ps = divs[3].find_all('p')
        kw_full = ps[3].text
        kw = kw_full.replace('Palavras-chave: ', '')
        kw = kw.replace(',', '.').replace(';', '.')
        kw = '. '.join([_.strip() for _ in kw.split('.')])
    except:
        kw = ''
    return {'Palavras-chave': kw}

In [16]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Gestão Ambiental. Contabilidade. Mineração. Relatório de Sustentabilidade. '}

In [17]:
def captura_autores(soup):
    autores = []
    try:
        content_tag = soup.find('div', {'id': 'content'})
        divs = content_tag.find_all('div')
        autores_full = divs[2].find('em')
        autores = autores_full.text.split(', ')
    except:
        pass

    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [18]:
captura_autores(soup)

{'Qtd. de Autores': 4,
 'Autor 1': 'Fernanda Beatriz Rolim Tavares',
 'Autor 2': 'Cristiane Queiroz Reis',
 'Autor 3': 'Janaina Ferreira Marques de Melo',
 'Autor 4': 'Fabiano Batista Ferreira',
 'Autor 5': '',
 'Autor 6': ''}

In [19]:
def captura_afiliacao(soup):
    affiliation_list = []
    try:
        authors_tag = soup.find_all('div', {'class': 'authorBio'})
        for p in authors_tag:
            try:
                affiliation = p.find_all('br')[-2].next_sibling
                affiliation = affiliation.replace('\t', '')
            except:
                affiliation = ''
            affiliation_list.append(affiliation)
    except:
        pass
    affiliation_list = completar_lista(affiliation_list)
    key_affiliation = [f"Afilição {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, affiliation_list))

In [20]:
captura_afiliacao(soup)

{'Afilição 1': '',
 'Afilição 2': '',
 'Afilição 3': '',
 'Afilição 4': '',
 'Afilição 5': '',
 'Afilição 6': ''}

In [21]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Pensar Contábil'}
    url_search_text = 'http://www.atena.org.br/revista/ojs-2.2.3-06/index.php/pensarcontabil/search/advancedResults?query=__query__&searchJournal=&author=&title=&fullText=&supplementaryFiles=&discipline=&subject=&type=&coverage=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'

    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        
        if 'Pré-textual' in captura_titulo(soup)['Título'] or 'CAP Accounting and Management' in captura_titulo(soup)['Título']:
            continue
            
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%Y')
    return df

## Pesquisa por artigos

In [22]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2004-01-01,A Contabilidade e sua Responsabilidade Social ...,Aborda sobre o ambiente como estratégia empres...,,1,Maria Elisabeth Pereira Kraemer,,,,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil
1,2002-01-01,Progressos no campo da Contabilidade aplicada ...,Discorre sobre a obrigatoriedade de contas esp...,,1,Antônio Lopes de Sá,,,,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil
2,2021-01-01,Análise de Indicadores de Desempenho da Gestão...,O desenvolvimento sustentável é requisito fund...,Gestão Ambiental. Contabilidade. Mineração. Re...,4,Fernanda Beatriz Rolim Tavares,Cristiane Queiroz Reis,Janaina Ferreira Marques de Melo,Fabiano Batista Ferreira,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil


In [23]:
df.to_csv('../data/PENSAR_CONTABIL.csv', index=False)

In [24]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afilição 1,Afilição 2,Afilição 3,Afilição 4,Afilição 5,Afilição 6,Url Artigo,Revista
0,2004-01-01,A Contabilidade e sua Responsabilidade Social ...,Aborda sobre o ambiente como estratégia empres...,,1,Maria Elisabeth Pereira Kraemer,,,,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil
1,2002-01-01,Progressos no campo da Contabilidade aplicada ...,Discorre sobre a obrigatoriedade de contas esp...,,1,Antônio Lopes de Sá,,,,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil
2,2021-01-01,Análise de Indicadores de Desempenho da Gestão...,O desenvolvimento sustentável é requisito fund...,Gestão Ambiental. Contabilidade. Mineração. Re...,4,Fernanda Beatriz Rolim Tavares,Cristiane Queiroz Reis,Janaina Ferreira Marques de Melo,Fabiano Batista Ferreira,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil
3,2021-01-01,"ESG, Impactos Ambientais e Contabilidade","A Agenda ESG (environmental, social and govern...",ESG. Sustentabilidade. ImpactosAmbientais. Tra...,1,Sebastião Bergamini Junior,,,,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil
4,2015-01-01,Heritage Assets no Brasil: um Estudo de Caso ...,Resumo Analisaram-se as principais normas inte...,,3,Esdras Carlos de Santana,Alex Sandro de Oliveira Barbosa,Larissa Gomes de Oliveira,,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil
5,2012-01-01,Importância do Balanço Social para as Entidade...,Key words: Social Report. Third Sector. Manage...,,4,Roberto Carlos Klann,Jorge Eduardo Scarpin,Silene Rengel,Paulo Roberto da Cunha,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil
6,2015-01-01,Adesão dos balanços sociais publicados pelos C...,,,4,Bruna Januário dos Santos,Denize Demarche Minatti Ferreira,Lisiane Adriana Martins,Luiz Felipe Ferreira,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil
7,2005-01-01,DEMONSTRAÇÃO DO VALOR ADICIONADO: A IMPORTÂNCI...,ABSTRACTThis article lifted foundations for re...,Key words: Added Value. Transparency. Social R...,3,Valter Roberto da Silva,Marcelo Erédia de Souza,Esliane Carecho B. Silva,,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil
8,2015-01-01,Adesão dos balanços sociais publicados pelos C...,,,4,Bruna Januário dos Santos,Denize Demarche Minatti Ferreira,Lisiane Adriana Martins,Luiz Felipe Ferreira,,,,,,,,,http://www.atena.org.br/revista/ojs-2.2.3-06/i...,Pensar Contábil
